In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Configure visualization if needed
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [5]:
# Load Starter Dataset
data_path = '../data/raw/'
unified_data = pd.read_csv(os.path.join(data_path, 'ethiopia_fi_unified_data.csv'))
impact_links = pd.read_csv(os.path.join(data_path, 'impact_links.csv'))
reference_codes = pd.read_csv(os.path.join(data_path, 'reference_codes.csv'))

print("Unified Data Shape:", unified_data.shape)
print("Impact Links Shape:", impact_links.shape)
print("Reference Codes Shape:", reference_codes.shape)

unified_data.head()


Unified Data Shape: (43, 34)
Impact Links Shape: (14, 35)
Reference Codes Shape: (71, 4)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


In [7]:
# 1. Explore Data Summary
print("--- Records by Type ---")
print(unified_data['record_type'].value_counts())

print("\n--- Records by Pillar ---")
print(unified_data['pillar'].value_counts())

print("\n--- Records by Source Type ---")
print(unified_data['source_type'].value_counts())

print("\n--- Temporal Range ---")
# Convert to datetime if possible, or just extract year
# unified_data['observation_date'] = pd.to_datetime(unified_data['observation_date']) # Format might be mixed
print(unified_data['observation_date'].min(), "to", unified_data['observation_date'].max())

print("\n--- Unique Indicators ---")
print(unified_data.loc[unified_data['record_type']=='observation', 'indicator_code'].unique())


--- Records by Type ---
record_type
observation    30
event          10
target          3
Name: count, dtype: int64

--- Records by Pillar ---
pillar
ACCESS           16
USAGE            11
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

--- Records by Source Type ---
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

--- Temporal Range ---
2014-12-31 to 2030-12-31

--- Unique Indicators ---
['ACC_OWNERSHIP' 'ACC_MM_ACCOUNT' 'ACC_4G_COV' 'ACC_MOBILE_PEN'
 'ACC_FAYDA' 'USG_P2P_COUNT' 'USG_P2P_VALUE' 'USG_ATM_COUNT'
 'USG_ATM_VALUE' 'USG_CROSSOVER' 'USG_TELEBIRR_USERS' 'USG_TELEBIRR_VALUE'
 'USG_MPESA_USERS' 'USG_MPESA_ACTIVE' 'USG_ACTIVE_RATE' 'AFF_DATA_INCOME'
 'GEN_GAP_ACC' 'GEN_MM_SHARE' 'GEN_GAP_MOBILE']


In [9]:
# 2. Enrichment Data Loading
enrich_A = pd.read_csv(os.path.join(data_path, 'enrichment_A_baselines.csv'))
enrich_B = pd.read_csv(os.path.join(data_path, 'enrichment_B_direct.csv'))
enrich_C = pd.read_csv(os.path.join(data_path, 'enrichment_C_indirect.csv'))

print("Enrichment A Columns:", enrich_A.columns.tolist())
print("Enrichment B Columns:", enrich_B.columns.tolist())
print("Enrichment C Columns:", enrich_C.columns.tolist())

# Show first few rows to understand structure
display(enrich_A.head(2))
display(enrich_B.head(2))


Enrichment A Columns: ['Unnamed: 0', 'Integrated Financial Access & Usage Index (IFAU index) ', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
Enrichment B Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
Enrichment C Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']


,Unnamed: 0,Integrated Financial Access & Usage Index (IFAU index),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Alternative Baseline Surveys,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,A,Alternative Baseline Surveys,NaN,NaN,NaN,NaN
1,B,Potential Direct Corelating Data Points,NaN,NaN,NaN,NaN


In [11]:
# 3. Check for existing values mentioned in Context
print("Ref checking:")
cols = ['indicator_code', 'observation_date', 'value_numeric', 'source_name']
print(unified_data.loc[unified_data['indicator_code'].isin(['USG_TELEBIRR_USERS', 'USG_MPESA_USERS', 'ACC_MM_ACCOUNT', 'USG_DIGITAL_PAYMENT']), cols])


Ref checking:
        indicator_code observation_date  value_numeric           source_name
6       ACC_MM_ACCOUNT       2021-12-31           4.70    Global Findex 2021
7       ACC_MM_ACCOUNT       2024-11-29           9.45    Global Findex 2024
20  USG_TELEBIRR_USERS       2025-06-30    54840000.00  Ethio Telecom Report
22     USG_MPESA_USERS       2024-12-31    10800000.00     Safaricom Results


In [13]:
# Check Account Ownership history
cols = ['indicator_code', 'observation_date', 'value_numeric', 'source_name']
print(unified_data.loc[unified_data['indicator_code'] == 'ACC_OWNERSHIP', cols].sort_values('observation_date'))


   indicator_code observation_date  value_numeric         source_name
0   ACC_OWNERSHIP       2014-12-31           22.0  Global Findex 2014
1   ACC_OWNERSHIP       2017-12-31           35.0  Global Findex 2017
2   ACC_OWNERSHIP       2021-12-31           46.0  Global Findex 2021
3   ACC_OWNERSHIP       2021-12-31           56.0  Global Findex 2021
4   ACC_OWNERSHIP       2021-12-31           36.0  Global Findex 2021
5   ACC_OWNERSHIP       2024-11-29           49.0  Global Findex 2024
30  ACC_OWNERSHIP       2025-12-31           70.0    NFIS-II Strategy


In [16]:
# 4. Enrich Dataset
print(unified_data.columns)

new_rows = []

# 1. 2011 Account Ownership
new_rows.append({
    'record_type': 'observation',
    'pillar': 'ACCESS',
    'indicator_code': 'ACC_OWNERSHIP',
    'indicator': 'Account ownership at a financial institution or with a mobile-money-service provider (% of adults)',
    'value_numeric': 14.0,
    'unit': 'Percentage',
    'observation_date': '2011-12-31',
    'source_name': 'Global Findex 2011',
    'source_type': 'survey',
    'source_url': 'https://www.worldbank.org/en/publication/globalfindex',
    'confidence': 'High',
    'notes': 'Added from Context overview description of trajectory.'
})

# 2. 2024 Digital Payment Usage
new_rows.append({
    'record_type': 'observation',
    'pillar': 'USAGE',
    'indicator_code': 'USG_DIGITAL_PAYMENT',
    'indicator': 'Made or received a digital payment (% of adults)',
    'value_numeric': 35.0,
    'unit': 'Percentage',
    'observation_date': '2024-11-29', # Same date as Findex 2024 in dataset
    'source_name': 'Global Findex 2024',
    'source_type': 'survey',
    'source_url': 'https://www.worldbank.org/en/publication/globalfindex', 
    'confidence': 'Medium', # Context says "~35%"
    'notes': 'Added from Context overview.'
})

# 3. 2024 Wages
new_rows.append({
    'record_type': 'observation',
    'pillar': 'USAGE',
    'indicator_code': 'USG_WAGES',
    'indicator': 'Used account to receive wages (% of adults)',
    'value_numeric': 15.0,
    'unit': 'Percentage',
    'observation_date': '2024-11-29',
    'source_name': 'Global Findex 2024',
    'source_type': 'survey',
    'source_url': 'https://www.worldbank.org/en/publication/globalfindex', 
    'confidence': 'Medium', # Context says "~15%"
    'notes': 'Added from Context overview.'
})

new_df = pd.DataFrame(new_rows)
# Align columns
for col in unified_data.columns:
    if col not in new_df.columns:
        new_df[col] = np.nan
        
# Ensure order matches
new_df = new_df[unified_data.columns]

enriched_data = pd.concat([unified_data, new_df], ignore_index=True)
print("New shape:", enriched_data.shape)
enriched_data.tail()


Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='object')
New shape: (46, 34)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
41,EVT_0009,event,policy,NaN,NFIS-II Strategy Launch,EVT_NFIS2,NaN,NaN,Launched,categorical,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,5-year national financial inclusion strategy,NaN
42,EVT_0010,event,pricing,NaN,Safaricom Ethiopia Price Increase,EVT_SAFCOM_PRICE,NaN,NaN,Implemented,categorical,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Data and voice prices increased 20-82%,NaN
43,NaN,observation,NaN,ACCESS,Account ownership at a financial institution o...,ACC_OWNERSHIP,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Added from Context overview description of tra...
44,NaN,observation,NaN,USAGE,Made or received a digital payment (% of adults),USG_DIGITAL_PAYMENT,NaN,35.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Added from Context overview.
45,NaN,observation,NaN,USAGE,Used account to receive wages (% of adults),USG_WAGES,NaN,15.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Added from Context overview.


In [17]:
# 5. Save Processed Data
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

enriched_data.to_csv(os.path.join(output_dir, 'ethiopia_fi_unified_data.csv'), index=False)
print(f"Saved enriched data to {os.path.join(output_dir, 'ethiopia_fi_unified_data.csv')}")

# Copy other files to processed for completeness
impact_links.to_csv(os.path.join(output_dir, 'impact_links.csv'), index=False)
reference_codes.to_csv(os.path.join(output_dir, 'reference_codes.csv'), index=False)
print("Saved impact_links and reference_codes to processed.")


Saved enriched data to ../data/processed/ethiopia_fi_unified_data.csv
Saved impact_links and reference_codes to processed.


# Task 1: Data Exploration and Enrichment

## Objective
Understand the starter dataset and enrich it with additional data found in `data/raw`.

## Data Sources
- Starter Dataset: `data/raw/ethiopia_fi_unified_data.csv`
- Impact Links: `data/raw/impact_links.csv`
- Reference Codes: `data/raw/reference_codes.csv`
- Additional Data Points: `data/raw/Additional Data Points Guide.xlsx - *.csv`
